In [ ]:
# Importieren der benötigten Bibliotheken

# Datenmanipulation
import pandas as pd

# Machine Learning
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.base import clone

# Warnings unterdrücken
import warnings
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action="ignore", category=DataConversionWarning)
warnings.filterwarnings(action="ignore", category=FutureWarning)

# Baseline Modellierung


In [2]:
# Einlesen der Trainings- und Testdaten
features_train = pd.read_csv("../data/processed/features_train.csv")
features_test = pd.read_csv("../data/processed/features_test.csv")
target_train = pd.read_csv("../data/processed/target_train.csv")
target_test = pd.read_csv("../data/processed/target_test.csv")

# Zielvariable in einem dict organisieren
targets = {
    "compressor": {
        "train": target_train.iloc[:, 0],
        "test": target_test.iloc[:, 0]
    },
    "turbine": {
        "train": target_train.iloc[:, 1],
        "test": target_test.iloc[:, 1]
    }
}

In [3]:
# Piplines für verschiedene Modelle
pipeline_lr = Pipeline([('scaler', StandardScaler()), ('model', LinearRegression())])
pipeline_rf = Pipeline([("model", RandomForestRegressor(random_state=42))])
pipeline_dt = Pipeline([("model", DecisionTreeRegressor(random_state=42))])

# Modelle in einem Dictionary organisieren
models = {
    "Linear Regression": pipeline_lr,
    "Random Forest": pipeline_rf,
    "Decision Tree": pipeline_dt
}

In [4]:
# Evaluierung der Baseline-Modelle mit allen Features
all_results = []
for target_name, target_data in targets.items():
    target_train_data = target_data["train"]
    target_test_data = target_data["test"]
    
    for model_name, model_pipeline in models.items():
        current_model = clone(model_pipeline)
        current_model.fit(features_train, target_train_data)
        target_pred = current_model.predict(features_test)
        
        r2 = r2_score(target_test_data, target_pred)
        mae = mean_absolute_error(target_test_data, target_pred)
        mse = mean_squared_error(target_test_data, target_pred)
        
        results = {"Target": target_name, "Model": model_name, "R2": r2, "MAE": mae, "MSE": mse}
        all_results.append(results)

results_df = pd.DataFrame(all_results)
results_df.to_csv("../data/results/baseline_results.csv", index=False)

print("--- Baseline Modellergebnisse ---")
print(results_df.round(4))

--- Baseline Modellergebnisse ---
       Target              Model      R2     MAE  MSE
0  compressor  Linear Regression  0.8383  0.0046  0.0
1  compressor      Random Forest  0.9956  0.0006  0.0
2  compressor      Decision Tree  0.9886  0.0010  0.0
3     turbine  Linear Regression  0.9072  0.0017  0.0
4     turbine      Random Forest  0.9910  0.0004  0.0
5     turbine      Decision Tree  0.9800  0.0005  0.0
